In [ ]:
pip uninstall autogen

In [ ]:
import chromadb
from typing_extensions import Annotated

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [ ]:
api_key = " "
azure_endpoint = " "
api_version = " "

llm_config = {"api_key": api_key, "api_version":api_version, "azure_endpoint":azure_endpoint, "model":" ", "api_type":"azure", "cache_seed": None}

In [ ]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

In [ ]:
topics_generator = autogen.UserProxyAgent(
    name="TopicsGenerator1",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    code_execution_config=False,  # we don't want to execute code in this case.
    description="An Agent which generates topics for the article that it is asked to write",
)

In [ ]:
examples_adder = AssistantAgent(
    name="ExamplesAdder",
    is_termination_msg=termination_msg,
    system_message="You read the json having an article written. Your job is to add more examples in that json. Add a new key Examples and then add atleast 4 examples to it. ",
    llm_config=llm_config,
    description="Adding more examples",
)

In [ ]:
advanced_topic = AssistantAgent(
    name="AdvancedTopic",
    is_termination_msg=termination_msg,
    system_message="You read the json having an article written. Your job is to add a new key which talks about advanced topics related to the article. Give final response in the same JSON format and then reply with TERMINATE",
    llm_config=llm_config,
    description="Adding a section on Advanced Topics",
)

In [ ]:
article_writer = autogen.AssistantAgent(
    name="ArticleWriter1",
    is_termination_msg=termination_msg,
    system_message="""You are a technical blog writer, you write an article on the topics provided by TopicsGenerator. Each topic should be written with a simple example. Make sure the final response that you give should be in JSON format so that it can be rendered in the UI. It should follow an example structure given below:
     \{
    "Title":"Unraveling Numpy Broadcasting With Clear Examples",
    "Body":\[
        \{
            "Content": "In the vibrant world of Python programming, Numpy stands as a fundamental library that empowers its users with extensive mathematical capabilities. Among these wonderful features of Numpy, a little appreciated yet pivotal one is 'Broadcasting'. This article will vividly explain and illustrate this fairly perplexing aspect of Numpy."
        \},
        \{
            "Heading": "What is Numpy Broadcasting?",
            "Content": "Numpy broadcasting is a powerful feature that allows us to perform arithmetic operations on arrays of different shapes. This could be between a scalar and an array, or between arrays of different shapes, under certain conditions."
        \},
        \{
            "Content": "The concept behind broadcasting pivots on two rules:",
            "List": \[
                \{
                    "ListItem": "If the arrays do not have the same rank, then a 1 will be pre-pended to the smaller ranking array’s shape."
                \}
            \]
        \},
        \{
            "Heading": "Python example of Numpy Broadcasting",
            "Content": "Let's illustrate this concept with a simple Python example.",
            "Code": \[
                \{
                    "CodeBlock": "import numpy as np\nA = np.array([1, 2, 3])\nB = np.array([2, 2, 2])\nC = A * B\nprint(C)"
                \},
                \{
                    "Content": "In this example, we're multiplying two arrays of identical shape, so broadcasting isn't necessary. However, observe what happens when we alter the second array:",
                    "CodeBlock": "B = 2\nC = A * B\nprint(C)"
                \}
            \]
        \}
    \]
\}""",
    llm_config=llm_config,
    description="technical blog writer who writes article on the list of topics provided",
)

In [ ]:
def _reset_agents():
    topics_generator.reset()
    article_writer.reset()
    examples_adder.reset()
    advanced_topic.reset()
    topics_generator.clear_history()
    article_writer.clear_history()
    examples_adder.clear_history()
    advanced_topic.clear_history()

In [ ]:
def create_article(PROBLEM):
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[topics_generator, article_writer, examples_adder, advanced_topic],
        messages=[],
        max_round=4,
        speaker_selection_method="round_robin",
        allow_repeat_speaker=False,
        enable_clear_history=True
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config, max_consecutive_auto_reply=1)
    topics_generator.initiate_chat(
        manager,
        message=PROBLEM,
    )

In [ ]:
import json
topics = ["Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."]
final_json = []
for i, topic in enumerate(topics):
    PROBLEM = f"Write an article similar to a blog on {topic}"
    create_article(PROBLEM)
    cleaned_json_string = topics_generator.last_message()["content"].replace("TERMINATE", "").strip()
    data = json.loads(cleaned_json_string)
    final_json.append(data)
    print(data)

In [ ]:
final = {}
for f in final_json:
    final = {**final, **f}